#### NLP Processing for our unstruvtured data
Now we have our four features but by observing above table we have some string data available and as we know that machines can not understand these strings so we have transform these string respresented data in to numerical form.

Here we can apply one hot encode to the subreddit but applying to title will not give proper result so for the title, we do nlp on that and trying to get one global matrix for title.

To do NLP on title, we have to follow the particular pipeline to get the major words from the title. We are applying pipeline given as below.

- REMOVE PUNCTUATION
- TOKENIZATION
- REMOVING STOPWORDS
- STEMMING (IF NEEDED TO IMPROVE ACCURACY)
- VECTORIZING

In [5]:
import pandas as pd
import os

In [6]:
def preprocess(df):
    
    df = df[df.columns.drop(list(df.filter(regex='^Cat')))]
    df = df[df['Date'] != '27/06/2001']  #removing the date
    df[(df['Subject'] != 'RE:') & (df['Subject'] != 'FW:') & (df['Subject'] != 'Re:')]  #removing the max same subjects
    del df['Unnamed: 0']
    return df

In [7]:
DATA = 'data' #https://data.world/brianray/enron-email-dataset

FILENAMES = [os.path.join(DATA, filename) for filename in os.listdir(DATA)]
df = pd.read_csv(FILENAMES[0])
df = preprocess(df)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
df.head()

,Message-ID,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user,labeled
0,<18782981.1075855378110.JavaMail.evans@thyme>,2001-05-14 23:39:00,frozenset({'phillip.allen@enron.com'}),frozenset({'tim.belden@enron.com'}),NaN,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast,allen-p,False
1,<15464986.1075855378456.JavaMail.evans@thyme>,2001-05-04 20:51:00,frozenset({'phillip.allen@enron.com'}),frozenset({'john.lavorato@enron.com'}),Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p,False
2,<24216240.1075855687451.JavaMail.evans@thyme>,2000-10-18 10:00:00,frozenset({'phillip.allen@enron.com'}),frozenset({'leah.arsdall@enron.com'}),Re: test,Phillip K Allen,Leah Van Arsdall,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p,False
3,<13505866.1075863688222.JavaMail.evans@thyme>,2000-10-23 13:13:00,frozenset({'phillip.allen@enron.com'}),frozenset({'randall.gay@enron.com'}),NaN,Phillip K Allen,Randall L Gay,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy, Can you send me a schedule of the salar...",allen-p,False
4,<30922949.1075863688243.JavaMail.evans@thyme>,2000-08-31 12:07:00,frozenset({'phillip.allen@enron.com'}),frozenset({'greg.piper@enron.com'}),Re: Hello,Phillip K Allen,Greg Piper,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p,False


In [9]:
dfEmail = df[['Subject', 'content']]

In [10]:
dfEmail.head()

,Subject,content
0,NaN,Here is our forecast
1,Re:,Traveling to have a business meeting takes the...
2,Re: test,test successful. way to go!!!
3,NaN,"Randy, Can you send me a schedule of the salar..."
4,Re: Hello,Let's shoot for Tuesday at 11:45.


##### Remove Punctuation
In this method, We are removing all the following the punctuations from content and subject

In [11]:

import string

print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [12]:
def remove_punc(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

''' python is case sensetive for that A and a is diffrent thats why lower()'''
dfEmail['content_removed_punctuation'] = dfEmail['content'].apply(lambda x: remove_punc(str(x).lower()))
dfEmail['Subject_removed_punctuation'] = dfEmail['Subject'].apply(lambda x: remove_punc(str(x).lower()))
dfEmail.head()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Subject,content,content_removed_punctuation,Subject_removed_punctuation
0,NaN,Here is our forecast,here is our forecast,nan
1,Re:,Traveling to have a business meeting takes the...,traveling to have a business meeting takes the...,re
2,Re: test,test successful. way to go!!!,test successful way to go,re test
3,NaN,"Randy, Can you send me a schedule of the salar...",randy can you send me a schedule of the salary...,nan
4,Re: Hello,Let's shoot for Tuesday at 11:45.,lets shoot for tuesday at 1145,re hello


We can see the difference in text by navigationg the columns

##### Tokenization
Tokenization is the act of breaking up a sequence of strings into pieces such as words, keywords, phrases, symbols and other elements called tokens. Tokens can be individual words, phrases or even whole sentences. In the process of tokenization, some characters like punctuation marks are discarded.

In [13]:
import re


def tokenize(text):
    # Split word non word
    tokens = re.split('\W+', text)
    return tokens

##EACH ROW
dfEmail['Subject_tokenize'] = dfEmail['Subject_removed_punctuation'].apply(lambda x: tokenize(x))
dfEmail['content_tokenize'] = dfEmail['content_removed_punctuation'].apply(lambda x: tokenize(x))
del dfEmail['content_removed_punctuation']
del dfEmail['Subject_removed_punctuation']
dfEmail.head()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,Subject,content,Subject_tokenize,content_tokenize
0,NaN,Here is our forecast,[nan],"[here, is, our, forecast]"
1,Re:,Traveling to have a business meeting takes the...,[re],"[traveling, to, have, a, business, meeting, ta..."
2,Re: test,test successful. way to go!!!,"[re, test]","[test, successful, way, to, go]"
3,NaN,"Randy, Can you send me a schedule of the salar...",[nan],"[randy, can, you, send, me, a, schedule, of, t..."
4,Re: Hello,Let's shoot for Tuesday at 11:45.,"[re, hello]","[lets, shoot, for, tuesday, at, 1145]"


We can see that each words in the sentence a

##### Remove StepWords
StepWords are those which are words which does not play importance in sentences. Like I am going to watch movie. where 'I', 'am' and 'to' plays part of the stopwords.

Here we will use nltk package to remvoe stopwords.

In [14]:
import nltk
print('NLTK (DOWNLOAD ALL PACKAGES TO PERFORM NLP OPERATION)')

print('UNCOMMENT FOLLOWING LINE To GET NLTK DOWNLOADED')
#nltk.download()
stopword = nltk.corpus.stopwords.words('english')

NLTK (DOWNLOAD ALL PACKAGES TO PERFORM NLP OPERATION)
UNCOMMENT FOLLOWING LINE To GET NLTK DOWNLOADED


In [15]:
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopword]
    return text

dfEmail['Subject_nostopwords'] = dfEmail['Subject_tokenize'].apply(lambda x: remove_stopwords(x))
dfEmail['content_nostopwords'] = dfEmail['content_tokenize'].apply(lambda x: remove_stopwords(x))
del dfEmail['Subject_tokenize']
del dfEmail['content_tokenize']
dfEmail.head()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Subject,content,Subject_nostopwords,content_nostopwords
0,NaN,Here is our forecast,[nan],[forecast]
1,Re:,Traveling to have a business meeting takes the...,[],"[traveling, business, meeting, takes, fun, tri..."
2,Re: test,test successful. way to go!!!,[test],"[test, successful, way, go]"
3,NaN,"Randy, Can you send me a schedule of the salar...",[nan],"[randy, send, schedule, salary, level, everyon..."
4,Re: Hello,Let's shoot for Tuesday at 11:45.,[hello],"[lets, shoot, tuesday, 1145]"


##### Stemming & Lemmatization

Stemming and Lemmatization are Text Normalization (or sometimes called Word Normalization) techniques in the field of Natural Language Processing that are used to prepare text, words, and documents for further processing. 

###### Stemming Algorithms and Code

 different stemmers available in different languages in Python nltk. For the English language, you can choose between <b>PorterStammer</b> or <b>LancasterStammer</b>, PorterStemmer being the oldest one originally developed in 1979. LancasterStemmer was developed in 1990 and uses a more aggressive approach than Porter Stemming Algorithm.

###### PorterStemmer

In [35]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

#create an object of class PorterStemmer
porter = PorterStemmer()
lancaster=LancasterStemmer()
#proide a word to be stemmed
print("Porter Stemmer")
print(porter.stem("cats"))
print(porter.stem("trouble"))
print(porter.stem("troubling"))
print(porter.stem("troubled"))
print("Lancaster Stemmer")
print(lancaster.stem("cats"))
print(lancaster.stem("trouble"))
print(lancaster.stem("troubling"))
print(lancaster.stem("troubled"))

Porter Stemmer
cat
troubl
troubl
troubl
Lancaster Stemmer
cat
troubl
troubl
troubl


In [17]:
def port_stem(tokenized_list):
    text = [porter.stem(word) for word in tokenized_list]
    return text

dfEmail['Subject_port_stem'] = dfEmail['Subject_nostopwords'].apply(lambda x: port_stem(x))
dfEmail['content_port_stem'] = dfEmail['content_nostopwords'].apply(lambda x: port_stem(x))


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Subject,content,Subject_nostopwords,content_nostopwords,Subject_port_stem,content_port_stem
0,NaN,Here is our forecast,[nan],[forecast],[nan],[forecast]
1,Re:,Traveling to have a business meeting takes the...,[],"[traveling, business, meeting, takes, fun, tri...",[],"[travel, busi, meet, take, fun, trip, especi, ..."
2,Re: test,test successful. way to go!!!,[test],"[test, successful, way, go]",[test],"[test, success, way, go]"
3,NaN,"Randy, Can you send me a schedule of the salar...",[nan],"[randy, send, schedule, salary, level, everyon...",[nan],"[randi, send, schedul, salari, level, everyon,..."
4,Re: Hello,Let's shoot for Tuesday at 11:45.,[hello],"[lets, shoot, tuesday, 1145]",[hello],"[let, shoot, tuesday, 1145]"


In [33]:
dfEmail[['Subject_nostopwords','Subject_port_stem', 'content_nostopwords','content_port_stem']].head(10)

,Subject_nostopwords,Subject_port_stem,content_nostopwords,content_port_stem
0,[nan],[nan],[forecast],[forecast]
1,[],[],"[traveling, business, meeting, takes, fun, tri...","[travel, busi, meet, take, fun, trip, especi, ..."
2,[test],[test],"[test, successful, way, go]","[test, success, way, go]"
3,[nan],[nan],"[randy, send, schedule, salary, level, everyon...","[randi, send, schedul, salari, level, everyon,..."
4,[hello],[hello],"[lets, shoot, tuesday, 1145]","[let, shoot, tuesday, 1145]"
5,[hello],[hello],"[greg, either, next, tuesday, thursday, phillip]","[greg, either, next, tuesday, thursday, phillip]"
6,[nan],[nan],"[please, cc, following, distribution, list, up...","[pleas, cc, follow, distribut, list, updat, ph..."
7,"[prc, review, phone, calls]","[prc, review, phone, call]","[morning, 10, 1130]","[morn, 10, 1130]"
8,"[high, speed, internet, access]","[high, speed, internet, access]","[1, login, pallen, pw, ke9davis, dont, think, ...","[1, login, pallen, pw, ke9davi, dont, think, r..."
9,"[fw, fixed, forward, collar, floor, gas, price...","[fw, fix, forward, collar, floor, ga, price, t...","[, forwarded, phillip, k, allenhouect, 1016200...","[, forward, phillip, k, allenhouect, 10162000,..."


###### LancasterStemmer


In [36]:
def lancaster_stem(tokenized_list):
    text = [lancaster.stem(word) for word in tokenized_list]
    return text

dfEmail['Subject_lancaster'] = dfEmail['Subject_nostopwords'].apply(lambda x: lancaster_stem(x))
dfEmail['content_lancaster'] = dfEmail['content_nostopwords'].apply(lambda x: lancaster_stem(x))


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [37]:
dfEmail[['Subject_nostopwords','Subject_lancaster', 'content_nostopwords','content_lancaster']].head(10)

,Subject_nostopwords,Subject_lancaster,content_nostopwords,content_lancaster
0,[nan],[nan],[forecast],[forecast]
1,[],[],"[traveling, business, meeting, takes, fun, tri...","[travel, busy, meet, tak, fun, trip, espec, pr..."
2,[test],[test],"[test, successful, way, go]","[test, success, way, go]"
3,[nan],[nan],"[randy, send, schedule, salary, level, everyon...","[randy, send, schedule, sal, level, everyon, s..."
4,[hello],[hello],"[lets, shoot, tuesday, 1145]","[let, shoot, tuesday, 1145]"
5,[hello],[hello],"[greg, either, next, tuesday, thursday, phillip]","[greg, eith, next, tuesday, thursday, phillip]"
6,[nan],[nan],"[please, cc, following, distribution, list, up...","[pleas, cc, follow, distribut, list, upd, phil..."
7,"[prc, review, phone, calls]","[prc, review, phon, cal]","[morning, 10, 1130]","[morn, 10, 1130]"
8,"[high, speed, internet, access]","[high, spee, internet, access]","[1, login, pallen, pw, ke9davis, dont, think, ...","[1, login, pal, pw, ke9davis, dont, think, req..."
9,"[fw, fixed, forward, collar, floor, gas, price...","[fw, fix, forward, coll, flo, gas, pric, term]","[, forwarded, phillip, k, allenhouect, 1016200...","[, forward, phillip, k, allenhouect, 10162000,..."


##### Lemmatization 

Lemmatization, unlike Stemming, reduces the inflected words properly ensuring that the root word belongs to the language. In Lemmatization root word is called Lemma. A lemma (plural lemmas or lemmata) is the canonical form, dictionary form, or citation form of a set of words

For example, runs, running, ran are all forms of the word run, therefore run is the lemma of all these words. Because lemmatization returns an actual word of the language, it is used where it is necessary to get valid words.

Python NLTK provides WordNet Lemmatizer that uses the WordNet Database to lookup lemmas of words.

<b>Note: Download the WordNet corpora from NLTK downloader before using the WordNet Lemmatizer<b>

In [40]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

sentence = "He was running and eating at same time. He has bad habit of swimming after playing long hours in the Sun."
punctuations="?:!.,;"
sentence_words = nltk.word_tokenize(sentence)
for word in sentence_words:
    if word in punctuations:
        sentence_words.remove(word)

sentence_words
print("{0:20}{1:20}".format("Word","Lemma"))
for word in sentence_words:
#     print ("{0:20}{1:20}".format(word,wordnet_lemmatizer.lemmatize(word))) this just removes plurals
    print ("{0:20}{1:20}".format(word,wordnet_lemmatizer.lemmatize(word, pos="v")))

Word                Lemma               
He                  He                  
was                 be                  
running             run                 
and                 and                 
eating              eat                 
at                  at                  
same                same                
time                time                
He                  He                  
has                 have                
bad                 bad                 
habit               habit               
of                  of                  
swimming            swim                
after               after               
playing             play                
long                long                
hours               hours               
in                  in                  
the                 the                 
Sun                 Sun                 


In [41]:
def lemma(tokenized_list):
    return [wordnet_lemmatizer.lemmatize(word, pos="v") for word in tokenized_list]   

dfEmail['Subject_lemma'] = dfEmail['Subject_nostopwords'].apply(lambda x: lemma(x))
dfEmail['content_lemma'] = dfEmail['content_nostopwords'].apply(lambda x: lemma(x))


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [42]:
dfEmail[['Subject_nostopwords','Subject_lemma', 'content_nostopwords','content_lemma']].head(10)

,Subject_nostopwords,Subject_lemma,content_nostopwords,content_lemma
0,[nan],[nan],[forecast],[forecast]
1,[],[],"[traveling, business, meeting, takes, fun, tri...","[travel, business, meet, take, fun, trip, espe..."
2,[test],[test],"[test, successful, way, go]","[test, successful, way, go]"
3,[nan],[nan],"[randy, send, schedule, salary, level, everyon...","[randy, send, schedule, salary, level, everyon..."
4,[hello],[hello],"[lets, shoot, tuesday, 1145]","[let, shoot, tuesday, 1145]"
5,[hello],[hello],"[greg, either, next, tuesday, thursday, phillip]","[greg, either, next, tuesday, thursday, phillip]"
6,[nan],[nan],"[please, cc, following, distribution, list, up...","[please, cc, follow, distribution, list, updat..."
7,"[prc, review, phone, calls]","[prc, review, phone, call]","[morning, 10, 1130]","[morning, 10, 1130]"
8,"[high, speed, internet, access]","[high, speed, internet, access]","[1, login, pallen, pw, ke9davis, dont, think, ...","[1, login, pallen, pw, ke9davis, dont, think, ..."
9,"[fw, fixed, forward, collar, floor, gas, price...","[fw, fix, forward, collar, floor, gas, price, ...","[, forwarded, phillip, k, allenhouect, 1016200...","[, forward, phillip, k, allenhouect, 10162000,..."


Now compare the above 3 steeming and lemmatization columns with the nostopwords column and analyze which technique is working best for us. It is still part of EDA but by going through the above port stemming, lancasster stemming and lemmatization columns, The lemmatization works as appropriate as it is not changing the meaning of the word and still truncating the extra characters

In [52]:

del dfEmail['content_lancaster']
del dfEmail['content_port_stem']
del dfEmail['Subject_lancaster']
del dfEmail['Subject_port_stem']

In [1]:
dfEmail.head()

NameError: name 'dfEmail' is not defined